In [1]:
import pandas as pd

In [2]:
import pandas as pd
import numpy as np

# Загрузка данных
df = pd.read_csv('buildings.csv')

# Создаем копию для работы
df = df.copy()

# Функция классификации
def classify_apartment(row):
    # Извлекаем признаки
    rooms = row.get('rooms', 0)
    hc_class = str(row.get('hc_class', '')).lower() if pd.notna(row.get('hc_class')) else ''
    min_metro = row.get('min_metro') if pd.notna(row.get('min_metro')) else 999
    price_per_m2 = row.get('price_per_m2', 0)
    total_area = row.get('total_area', 0)
    
    # Получаем общее количество этажей
    floor_total = row.get('floor_total', 1)
    
    # Определяем категории признаков
    # Количество комнат
    is_rooms_0_1 = rooms in [0, 1]
    is_rooms_2_3 = rooms in [2, 3]
    is_rooms_1_2 = rooms in [1, 2]
    is_rooms_3plus = rooms >= 3
    
    # Класс ЖК
    is_comfort = any(word in hc_class for word in ['комфорт'])
    is_business = any(word in hc_class for word in ['бизнес'])
    is_premium = any(word in hc_class for word in ['премиум'])
    is_business_premium = is_business or is_premium
    
    # Близость к метро (в минутах)
    is_very_close_metro = min_metro <= 5  # Очень близко (0-5 мин)
    is_close_metro = min_metro <= 10  # Близко (6-10 мин)
    is_medium_metro = 10 < min_metro <= 20  # Средне (11-20 мин)
    is_far_metro = min_metro > 20  # Далеко
    
    # Цена за м2 (определяем квартили)
    price_q1 = df['price_per_m2'].quantile(0.25)
    price_q2 = df['price_per_m2'].quantile(0.50)
    price_q3 = df['price_per_m2'].quantile(0.75)
        
    is_very_low_price = price_per_m2 < price_q1  # Ниже 25%
    is_low_price = price_q1 <= price_per_m2 < price_q2  # 25-50%
    is_medium_price = price_q2 <= price_per_m2 <= price_q3  # 50-75%
    is_high_price = price_per_m2 > price_q3  # Выше 75%
    
    # Площадь квартиры
    is_small_area = total_area < 50  # Маленькая
    is_medium_area = 50 <= total_area <= 80  # Средняя
    is_large_area = total_area > 80  # Большая
    
    # Дополнительные признаки
    description = str(row.get('description', '')).lower()
    is_city = row.get('is_city', False)
    is_popular_dev = row.get('popular_developer', False)
    has_parking = row.get('parking', False)
    has_parks = row.get('parks', False)
    is_available = row.get('available', False)
    is_ready = row.get('ready', False)
    has_court = row.get('court', False)
    has_entrance = row.get('entrance', False)
    
    # Ключевые слова в описании для каждого кластера
    has_optimal_value_words = any(word in description for word in [
        'выгодн', 'оптимальн', 'соотношен', 'цена-качество', 'бюджетн',
        'доступн', 'экономичн', 'разумн', 'практичн', 'функционал'
    ])
    
    has_district_words = any(word in description for word in [
        'район', 'инфраструктур', 'магазин', 'супермаркет', 'школ',
        'детск', 'поликлин', 'больниц', 'аптек', 'бытов', 'услуг'
    ])
    
    has_perfect_place_words = any(word in description for word in [
        'идеальн', 'отличн', 'прекрасн', 'баланс', 'гармони',
        'тих', 'спокойн', 'уединен', 'экологич', 'зелен'
    ])
    
    has_modern_words = any(word in description for word in [
        'современн', 'технолог', 'инновац', 'умный дом', 'дизайн',
        'архитектур', 'премиальн', 'эксклюзив', 'клубн'
    ])
    
    has_hotel_words = any(word in description for word in [
        'апартамент', 'отель', 'гостиничн', 'временн', 'краткосрочн',
        'сервис', 'обслуживан', 'консьерж', 'ресепшен', 'номер'
    ])
    
    # Логика классификации
    
    # Кластер 4 (Временное проживание/апартаменты)
    # rooms 0-1, Бизнес/Премиум, очень близко к метро, высокая цена
    if is_rooms_0_1 and (is_business_premium or has_hotel_words) and is_very_close_metro:
        confirmations = 0
        if has_hotel_words:
            confirmations += 2
        if is_high_price:
            confirmations += 1
        if 'апартамент' in str(row.get('residential_complex', '')).lower():
            confirmations += 2
            
        if confirmations >= 2:
            return 4
    
    # Кластер 3 (Гармоничный район для жизни)
    # rooms 2-3, Комфорт, средняя/низкая цена, есть инфраструктура
    if is_rooms_2_3 and is_comfort and (is_low_price or is_medium_price):
        confirmations = 0
        if is_medium_metro or is_close_metro:
            confirmations += 1
        if has_district_words:
            confirmations += 2  # Важно для района
            
        if confirmations >= 3:
            return 3
    
    # Кластер 0 (Оптимальное соотношение цены и качества)
    # rooms 1-2, Комфорт, средняя цена, хорошая транспортная доступность
    if is_rooms_1_2 and is_comfort and (is_medium_price or is_low_price):
        confirmations = 0
        
        # Проверяем наличие преимуществ без переплаты
        advantages_count = 0
        if is_close_metro or is_medium_metro:
            advantages_count += 1
        
        if has_optimal_value_words:
            confirmations += 2
        
        # Проверяем, что не слишком дешево (не эконом) и не слишком дорого
        if is_medium_price:
            confirmations += 1
        
        # Площадь должна быть средней (не слишком маленькая, не слишком большая)
        if is_medium_area:
            confirmations += 1
        
        if confirmations >= 2:
            return 0
    
    # Кластер 1 (Современный жилой комплекс)
    # Бизнес/Премиум, высокая цена, современные технологии
    if is_business_premium and is_high_price:
        confirmations = 0
        if has_modern_words:
            confirmations += 2
        if floor_total > 10:
            confirmations += 1
            
        if confirmations >= 3:
            return 1
    
    # 5. Кластер 2 (Идеальное место)
    # Очень близко к метро, средняя/высокая цена, акцент на качестве жизни
    if is_very_close_metro or is_close_metro:
        confirmations = 0
        if has_perfect_place_words:
            confirmations += 2
        if is_medium_price or is_high_price:
            confirmations += 1
            
        if confirmations >= 3:
            return 2
    
    # Резервная логика по основным правилам
    # Приоритет 1: По количеству комнат
    if is_rooms_0_1:
        if is_business_premium or has_hotel_words:
            return 4
        else:
            return 0  # Оптимальное соотношение для студий/1-комнатных
    
    elif is_rooms_2_3:
        if is_comfort and (is_low_price or is_medium_price):
            return 3  # Гармоничный район
        elif is_business_premium:
            return 1  # Современный комплекс
    
    elif is_rooms_1_2:
        if is_comfort:
            return 0  # Оптимальное соотношение
        elif is_business_premium:
            return 1  # Современный комплекс
    
    elif is_rooms_3plus:
        if is_comfort:
            return 3  # Гармоничный район для семей
        else:
            return 1  # Премиальные многокомнатные
    
    # Приоритет 2: По классу и цене
    if is_business_premium:
        if is_high_price:
            return 1
        elif is_very_close_metro:
            return 4
        else:
            return 2
    
    elif is_comfort:
        if is_low_price or is_medium_price:
            if is_rooms_2_3:
                return 3
            else:
                return 0
        else:
            return 0
    
    # Приоритет 3: По близости к метро
    if is_very_close_metro:
        return 2  # Идеальное место
    elif is_close_metro:
        return 0  # Оптимальное соотношение
    elif is_medium_metro:
        return 0  # Оптимальное соотношение
    elif is_far_metro:
        return 3  # Гармоничный район на окраине
    
    # Приоритет 4: По цене
    if is_high_price:
        return 1
    elif is_medium_price:
        return 0
    elif is_low_price:
        return 3
    
    # По умолчанию - оптимальное соотношение
    return 0

# Применяем классификацию
df['cluster'] = df.apply(classify_apartment, axis=1)

# Обновляем описание кластера 0
cluster_descriptions = {
    0: 'Оптимальное соотношение цены и качества: максимум преимуществ без переплаты для людей со средним достатком',
    1: 'Современный жилой комплекс: технологичный проект как законченный продукт',
    2: 'Идеальное место: безупречное место с идеальным балансом доступности и качества среды',
    3: 'Гармоничный район: сформированный район для жизни "здесь и сейчас"',
    4: 'Временное проживание/апартаменты: безупречно комфортное проживание без хлопот'
}

df['cluster_description'] = df['cluster'].map(cluster_descriptions)

# Сохраняем результат
df.to_csv('buildings_classified_optimal_value2.csv', index=False, encoding='utf-8-sig')

cluster_stats = df['cluster'].value_counts().sort_index()
for cluster_id, count in cluster_stats.items():
    percentage = (count / len(df)) * 100
    print(f"Кластер {cluster_id}: {count} объектов ({percentage:.1f}%)")

Кластер 0: 643 объектов (45.3%)
Кластер 1: 144 объектов (10.1%)
Кластер 2: 150 объектов (10.6%)
Кластер 3: 122 объектов (8.6%)
Кластер 4: 360 объектов (25.4%)
